In [67]:
pip install yfinance


In [68]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web

# CAPM MODEL

In [69]:
#CAPM
RISKY_ASSET = 'AMZN'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2018-01-01'

df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 adjusted=True,
                 progress=False)
X = df['Adj Close'].rename(columns={RISKY_ASSET: 'asset', 
                                    MARKET_BENCHMARK: 'market'}) \
                   .resample('M') \
                   .last() \
                   .pct_change() \
                   .dropna()
y = X.pop('asset')

# add constant
X = sm.add_constant(X)

# define and fit the regression model 
capm_model = sm.OLS(y, X).fit()

# print results 
print('CAPM MODEL SUMMARY\n',capm_model.summary())

CAPM MODEL SUMMARY
                             OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     28.77
Date:                Fri, 04 Dec 2020   Prob (F-statistic):           6.31e-06
Time:                        13:08:04   Log-Likelihood:                 47.065
No. Observations:                  35   AIC:                            -90.13
Df Residuals:                      33   BIC:                            -87.02
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0166      0.011

# FAMA FRENCH FIVE FACTOR MODEL

In [70]:
# five factors
df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3', 
                                'famafrench', 
                                start=START_DATE)[0]
df_five_factor.index = df_five_factor.index.format()

In [71]:
asset_df = yf.download(RISKY_ASSET,
                       start=START_DATE,
                       adjusted=True,
                       progress=False)
y = asset_df['Adj Close'].resample('M') \
                         .last() \
                         .pct_change() \
                         .dropna()

y.index = y.index.strftime('%Y-%m')
y.name = 'return'
y= pd.DataFrame(y)
# join all datasets on the index
five_factor_data = pd.merge(y, df_five_factor, left_index=True, right_index=True)
five_factor_data.columns = [ 'rtn','mkt', 'smb', 'hml', 'rmw', 'cma', 'rf']
five_factor_data.loc[:, five_factor_data.columns != 'rtn'] /= 100
# calculate excess returns
five_factor_data['excess_rtn'] = five_factor_data.rtn - five_factor_data.rf

five_factor_model = smf.ols(
    formula='excess_rtn ~ mkt + smb + hml + rmw + cma', 
    data=five_factor_data
).fit()

print('FAMA FRENCH FIVE FACTOR MODEL\n',five_factor_model.summary())




FAMA FRENCH FIVE FACTOR MODEL
                             OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     9.840
Date:                Fri, 04 Dec 2020   Prob (F-statistic):           1.93e-05
Time:                        13:08:04   Log-Likelihood:                 50.154
No. Observations:                  33   AIC:                            -88.31
Df Residuals:                      27   BIC:                            -79.33
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0018